# Imports & Installs

In [ ]:
#'''
!pip install contractions
!pip install ftfy
#'''
print("Done")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
from ftfy import fix_text
print("Done")

Done


In [4]:
#"""
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nlp = spacy.load("en_core_web_sm")

#"""

print("Done")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Done


In [5]:
file_path = "/content/drive/MyDrive/Grad Project/data_merged.csv"
df = pd.read_csv(file_path)
df.head()

,claim_status,video_transcription_text
0,opinion,my colleagues are willing to say that one-thir...
1,Supported claim,A report from Reuters revealed that WASHINGTON...
2,Supported claim,Reuters said that WASHINGTON U.S. House of Rep...
3,Supported claim,US Census Bureau confirmed that found that soc...
4,claim,which abuts the Sahara to the north and has be...


# Text Preprocessing

In [6]:
# Reference keywords
REFERENCE_KEYWORDS = [
    "according to", "reported that", "confirmed that", "stated that",
    "highlighted that", "emphasized that", "revealed that", "a study by",
    "research from", "in its latest report"
]

# Subjectivity markers
SUBJECTIVITY_MARKERS = [
    "i","i think", "in my opinion", "seems", "probably", "maybe", "feel", "i believe"
]

In [7]:
# Phase 1: Fixing Contrations

df["phase1_fixed"] = df["video_transcription_text"].apply(lambda x: fix_text(str(x)))

df[["video_transcription_text", "phase1_fixed"]].head()

,video_transcription_text,phase1_fixed
0,my colleagues are willing to say that one-thir...,my colleagues are willing to say that one-thir...
1,A report from Reuters revealed that WASHINGTON...,A report from Reuters revealed that WASHINGTON...
2,Reuters said that WASHINGTON U.S. House of Rep...,Reuters said that WASHINGTON U.S. House of Rep...
3,US Census Bureau confirmed that found that soc...,US Census Bureau confirmed that found that soc...
4,which abuts the Sahara to the north and has be...,which abuts the Sahara to the north and has be...


In [8]:
# Phase 2: Expand contractions

df["phase2_contractions"] = df["phase1_fixed"].apply(lambda x: contractions.fix(str(x)))


df[["phase1_fixed", "phase2_contractions"]].head()

,phase1_fixed,phase2_contractions
0,my colleagues are willing to say that one-thir...,my colleagues are willing to say that one-thir...
1,A report from Reuters revealed that WASHINGTON...,A report from Reuters revealed that WASHINGTON...
2,Reuters said that WASHINGTON U.S. House of Rep...,Reuters said that WASHINGTON YOU.S. House of R...
3,US Census Bureau confirmed that found that soc...,US Census Bureau confirmed that found that soc...
4,which abuts the Sahara to the north and has be...,which abuts the Sahara to the north and has be...


In [9]:
changed_rows = df[df["phase1_fixed"] != df["phase2_contractions"]]
changed_rows[["phase1_fixed", "phase2_contractions"]].head(10)

,phase1_fixed,phase2_contractions
0,my colleagues are willing to say that one-thir...,my colleagues are willing to say that one-thir...
1,A report from Reuters revealed that WASHINGTON...,A report from Reuters revealed that WASHINGTON...
2,Reuters said that WASHINGTON U.S. House of Rep...,Reuters said that WASHINGTON YOU.S. House of R...
7,Republican presidential nominee Donald Trump u...,Republican presidential nominee Donald Trump u...
9,up former Mexico City mayor Andres Manuel Lope...,up former Mexico City mayor Andres Manuel Lope...
11,I don't buy into the idea that lightning never...,I do not buy into the idea that lightning neve...
17,directed unwittingly to a messaging app linked...,directed unwittingly to a messaging app linked...
19,Reuters mentioned that ROME Italian Prime Mini...,Reuters mentioned that ROME Italian Prime Mini...
21,"Personally, lightning never strikes the same p...","Personally, lightning never strikes the same p..."
25,my friends are willing to wager that cats can'...,my friends are willing to wager that cats cann...


In [10]:
# Phase 3: cleaning

def clean_special_chars(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # remove urls
    text = re.sub(r"[^a-z\s]", " ", text)               # remove non-letters
    return text.strip()

df["phase3_cleaned"] = df["phase2_contractions"].apply(clean_special_chars)

changed_rows = df[df["phase2_contractions"] != df["phase3_cleaned"]]
changed_rows[["phase2_contractions", "phase3_cleaned"]].head()

,phase2_contractions,phase3_cleaned
0,my colleagues are willing to say that one-thir...,my colleagues are willing to say that one thir...
1,A report from Reuters revealed that WASHINGTON...,a report from reuters revealed that washington...
2,Reuters said that WASHINGTON YOU.S. House of R...,reuters said that washington you s house of r...
3,US Census Bureau confirmed that found that soc...,us census bureau confirmed that found that soc...
4,which abuts the Sahara to the north and has be...,which abuts the sahara to the north and has be...


In [11]:
# Phase 4: lem

from IPython.display import display, HTML
display(HTML("<h4 style='color:#00E5FF;'>Cell Running…</h4>"))

def spacy_process(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc]
    return {"lemmas": " ".join(tokens)}

df["phase4_spacy"] = df["phase3_cleaned"].apply(spacy_process)

display(HTML("<h4 style='color:#00FF8C;'> Done!</h4>"))


In [12]:
# Expand results for readability
df_spacy = pd.json_normalize(df["phase4_spacy"])
df_final = pd.concat([df, df_spacy], axis=1)

df_final[["phase3_cleaned", "lemmas"]].head(-30)

,phase3_cleaned,lemmas
0,my colleagues are willing to say that one thir...,my colleague be willing to say that one third ...
1,a report from reuters revealed that washington...,a report from reuter reveal that washington de...
2,reuters said that washington you s house of r...,reuters say that washington you s house of r...
3,us census bureau confirmed that found that soc...,us census bureau confirm that find that social...
4,which abuts the sahara to the north and has be...,which abut the sahara to the north and have be...
...,...,...
47241,as reported by reuters washington the national...,as report by reuters washington the national f...
47242,this blew my mind at the first modern olympic...,this blow my mind at the first modern olympi...
47243,you probably do not know this but the largest...,you probably do not know this but the large ...
47244,looking people said the arizona lawmaker wh...,look people say the arizona lawmaker who ...


In [13]:
df_export = df_final[[
    "claim_status",      # target labels
    "phase3_cleaned",    # baseline input
    "lemmas"            # main ML input
]].copy()

df_export.to_csv("preprocessed_2.csv", index=False)


In [14]:
print(df_export.shape)

(47276, 3)


In [15]:
df_export.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47276 entries, 0 to 47275
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   claim_status    47276 non-null  object
 1   phase3_cleaned  47276 non-null  object
 2   lemmas          47276 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [16]:
df=pd.read_csv("preprocessed_2.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47276 entries, 0 to 47275
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   claim_status    47276 non-null  object
 1   phase3_cleaned  47274 non-null  object
 2   lemmas          47274 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [17]:
print(df.duplicated().sum())

3279


In [18]:
df=df.drop_duplicates()
print(df.duplicated().sum())

0


In [19]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 43997 entries, 0 to 47275
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   claim_status    43997 non-null  object
 1   phase3_cleaned  43996 non-null  object
 2   lemmas          43996 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB
None


In [20]:
df.to_csv("preprocessed_2.csv", index=False)

# References